<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Знакомство-с-данными" data-toc-modified-id="Знакомство-с-данными-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Знакомство с данными</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработка данных</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#LightGBM" data-toc-modified-id="LightGBM-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>LightGBM</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Определение токсичных комментариев для «Викишоп»

## Подготовка

Импортируем необходимые библиотеки. 

In [1]:
import pandas as pd
import re
import spacy
from nltk.corpus import stopwords

from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import f1_score

from sklearn.dummy import DummyClassifier

from tqdm import tqdm

rs=1923

### Знакомство с данными

Считаем файл с данными. 

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Нам дан датасет, в котором 159292 записей. Один из столбцов, Unnamed: 0, не несет в себе полезной информации. 

In [4]:
data.sample(5)

,Unnamed: 0,text,toxic
153484,153641,The British resistance was broken and the sett...,0
7005,7013,Who exactly did she apologise to? The Comment...,0
68208,68276,{{unblock|Then you should also block Libro0 si...,0
56606,56667,hi \n\nyou are a fucking bitch,1
140586,140738,That's what you're heavily implying in linking...,0


Посмотрим, как распределены классы. 

In [5]:
data['toxic'].value_counts(normalize=True)

toxic
0    0.898388
1    0.101612
Name: proportion, dtype: float64

В данных присутствует дисбаланс классов, положительный класс составляет всего 10% данных. 

### Предобработка данных

Для начала с помощью регулярных выражений очистим текст от нерелевантных символов, таких как знаки препинания, спец. символы и тд. Оставим только символы английского алфавита. 

In [6]:
def clear_text(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    return ' '.join(text.lower().split())

In [7]:
data['text_processed'] = data['text'].apply(clear_text)

In [8]:
data.sample(5)

,Unnamed: 0,text,toxic,text_processed
46656,46711,"""\n\n LAN A340 solely for Sydney? \n\nThis art...",0,lan a solely for sydney this article says that...
93856,93948,1248277865155 define recognition]? Lately it's...,0,define recognition lately it s been kind of wa...
142326,142479,"""\n\n Sodomy \n\nHave you thought about creati...",1,sodomy have you thought about creating a wikip...
146322,146478,"""\n\n Please do not vandalize pages, as you di...",0,please do not vandalize pages as you did with ...
129925,130061,Fixed Up article\nI Just fixed up her discogra...,0,fixed up article i just fixed up her discograp...


Теперь удалим стоп-слова (слова, не несущие смысла). 

In [9]:
stop_words = stopwords.words('english')

In [10]:
def remove_stop_words(text):
    return " ".join([word for word in text.split() if word not in stop_words])

In [11]:
data['text_processed'] = data['text_processed'].apply(remove_stop_words)

In [12]:
data.sample(5)

,Unnamed: 0,text,toxic,text_processed
87791,87872,I thought about this some more and went ahead ...,0,thought went ahead emailed talk
66957,67024,a gentle warning \n\nBy now it should be evide...,0,gentle warning evident many editors watching b...
122217,122322,"""\n\n""""Nice try""""? What a silly statement. I h...",0,nice try silly statement violated policies nea...
38096,38142,It's true too. See the top of my talk page -_-...,0,true see top talk page user talk mistress seli...
43766,43819,President Jackson in 1760 ???? -please check y...,0,president jackson please check history


Лемматизируем тексты. Обработку будем проводить с помощью библиотеки `spiCy`, в которой лемматизация и POS-теггинг интегрированы в стандартный пайплайн обработки текста, то есть при ее использовании не нужно явно указывать части речи для корректной лемматизации. Также для эффективной обработки применим `nlp.pipe`, который обрабатывает тексты пакетами и позволяет "отключать" ненужные компоненты пайплайна. 

In [13]:
nlp = spacy.load('en_core_web_sm')

In [14]:
def spacy_lemmatize_pipe(texts):
    lemmas_texts = []
    for doc in tqdm(nlp.pipe(texts, disable=["parser", "ner"]), total=len(texts)):
        lemmas_texts.append(' '.join([token.lemma_ for token in doc]))
    return lemmas_texts

In [15]:
data['text_processed'] = spacy_lemmatize_pipe(data['text_processed'].tolist())

100%|█████████████████████████████████████████████████████████████████████████| 159292/159292 [04:03<00:00, 654.11it/s]


In [16]:
data.sample(5)

,Unnamed: 0,text,toxic,text_processed
25888,25912,Blocked\nIt would appear from your edit histor...,0,block would appear edit history use sort autom...
100625,100722,Your comments on Ireland (state) talk (whateve...,1,comment ireland state talk whatever call frank...
92799,92891,I am coming to get you \n\ni know where you li...,1,come get know live I m go come rape
147575,147731,On account of the project deciding to ignore h...,1,account project decide ignore history hereby q...
6434,6438,Would we still be able to receive the previous...,0,would still able receive previous article write


Оставим в данных только информативные столбцы.

In [17]:
data = data[['text_processed', 'toxic']]

In [18]:
data.head()

,text_processed,toxic
0,explanation edit make username hardcore metall...,0
1,aww match background colour seemingly stuck th...,0
2,hey man really try edit war guy constantly rem...,0
3,make real suggestion improvement wonder sectio...,0
4,sir hero chance remember page,0


## Обучение

Разделим данные на обучающую и тестовую выборки. 

In [19]:
train, test = train_test_split(data, test_size=0.25, random_state=rs)

Воспользуемся счетчиком величин TF-IDF из sklearn, создадим объект, который затем будем использовать внутри  `Pipeline`. Это необходимо, чтобы на кросс-валидации не происходило утечки данных из валидационной выборки. 

In [20]:
tf_idf = TfidfVectorizer()

features_train = train['text_processed']
target_train = train['toxic']

На следующем этапе с помощью кросс-валидации обучим модели Логистической регрессии, Случайного леса и LightGBM. Для всех трех моделей укажем, что выборка несбалансирована. 

### Логистическая регрессия

In [21]:
pipeline = Pipeline([('tfidf', tf_idf), ('lr', LogisticRegression(random_state=rs, max_iter=1000, class_weight='balanced'))])

param_grid = {
    'lr__solver' : ['lbfgs', 'liblinear'],
    'lr__C': [0.001, 0.01, 0.1, 1] + list(range(5, 16))
}

lr_grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=3, n_jobs=-1, scoring='f1')

lr_grid_search.fit(features_train, target_train)

print(f"Параметры лучшей модели логистической регрессии : {lr_grid_search.best_params_}")
print(f"F1 для лучшей модели логистической регрессии : {lr_grid_search.best_score_}")

Параметры лучшей модели логистической регрессии : {'lr__C': 11, 'lr__solver': 'liblinear'}
F1 для лучшей модели логистической регрессии : 0.7643893420244171


### Случайный лес

In [22]:
pipeline = Pipeline([('tfidf', tf_idf), ('rf', RandomForestClassifier(random_state=rs, class_weight='balanced'))])

param_grid = {
    'rf__n_estimators': [10, 50, 100],
    'rf__max_depth': [5, 10, 20],
    'rf__min_samples_split': [2, 5, 10]
}
    
rf_grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=3, n_jobs=-1, scoring='f1')

rf_grid_search.fit(features_train, target_train)

print(f"Параметры лучшей модели случайного леса: {rf_grid_search.best_params_}")
print(f"F1 для лучшей модели случайного леса: {rf_grid_search.best_score_}")

Параметры лучшей модели случайного леса: {'rf__max_depth': 20, 'rf__min_samples_split': 2, 'rf__n_estimators': 100}
F1 для лучшей модели случайного леса: 0.40677642695030225


### LightGBM

In [23]:
pipeline = Pipeline([('tfidf', tf_idf), ('lgbm', LGBMClassifier(random_state=rs, verbose=-1, is_unbalance=True))])

param_grid = {
    'lgbm__n_estimators': [10, 50, 100],
    'lgbm__max_depth': [5, 10, 20],
    'lgbm__learning_rate': [0.03, 0.1]
}

lgbm_grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=3, n_jobs=-1, scoring='f1')

lgbm_grid_search.fit(features_train, target_train)

print(f"Параметры лучшей модели LightGBM: {lgbm_grid_search.best_params_}")
print(f"F1 для лучшей модели LightGBM: {lgbm_grid_search.best_score_}")

Параметры лучшей модели LightGBM: {'lgbm__learning_rate': 0.1, 'lgbm__max_depth': 20, 'lgbm__n_estimators': 100}
F1 для лучшей модели LightGBM: 0.7377521246925239


### Вывод

На данном этапе была проведена подготовка данных и обучение трех различных моделей машинного обучения для классификации тональности текста на основе векторизации TF-IDF: Логистической регрессии, Случайного леса и LightGBM.  

Результаты (метрика F1):
- Логистическая регрессия : 0.764
- Случайный лес : 0.407
- LightGBM : 0.738

Наилучший результат на кросс-валидации показала модель Логистической регрессии. На втором месте LightGBM. Модель Случайного леса показала худший результат. 

## Тестирование

Используем найденную на кросс-валидации лучшую модель для тестирования на тестовых данных. 

In [24]:
best_model = lr_grid_search.best_estimator_

fitted_tf_idf = best_model.named_steps['tfidf']
best_logreg_model = best_model.named_steps['lr']

In [25]:
tf_idf_test = fitted_tf_idf.transform(test['text_processed'])
target_test = test['toxic']

In [26]:
f1_score(target_test, best_logreg_model.predict(tf_idf_test))

0.764933000227118

Показатель метрики F1 на тестовых данных равен 0.765, что удовлетворяет требованиям.

Проверим модель на адекватность путем сравнения показателя с показателем базовой (Dummy) модели. 

In [27]:
dummy_model = DummyClassifier(strategy='most_frequent', random_state=rs)
dummy_model.fit(fitted_tf_idf.transform(features_train), target_train)
f1_score(target_test, dummy_model.predict(tf_idf_test))

0.0

F1-метрика дамми-модели равна 0, так как модель никогда не предсказывает положительный класс. 

Чтобы лучше понять, насколько адекватна наша модель, сравним показатели F1-macro, эта метрика показывает среднее по метрикам F1 для каждого класса. 

In [28]:
f1_score(target_test, best_logreg_model.predict(tf_idf_test), average='macro')

0.8678561105031695

In [29]:
f1_score(target_test, dummy_model.predict(tf_idf_test), average='macro')

0.47296885959688195

Показатель среднего значения F1 для обученной модели равен 0.868, для дамми-модели 0.472, следовательно, проверка на адекватность пройдена. 

## Выводы

В ходе исследования были выполнены следующие шаги:

1. Изучение данных: 
    - Дан датасет с 159292 записями, хранящий текст комментария и маркер "токсичности" комментария
2. Предобработка данных. На данном этапе были проведены :
    - Очистка текста от ненужных символов
    - Приведение к нижнему регистру
    - Удаление стоп-слов
    - Лемматизация
3. Обучение моделей
    - Разделение данных на обучающую и тестовую выборку
    - Была выполнена TF-IDF векторизация
    - С помощью кросс-валидации были обучены три модели (Логистическая регрессия, Случайный лес, LightGBM) и определены лучшие значения гиперпараметров 
4. Оценка моделей 
    - Наилучший результат на кросс-валидации показала модель Логистической регрессии с метрикой F1 около 0.764, в то время как модель Случайного леса показала худший результат с метрикой F1 равной 0.407, и LightGBM показала результат сопоставимый с Логистической регрессией — 0.738.
5. Тестирование 
    - Модель Логистической регресии с найденными лучшими значениниями гиперпараметров была проверена на адекватность путем сравнения с базовой моделью. Dummy-модель не смогла предсказать положительный класс (F1-метрика равна 0). Использование метрики F1-macro подтвердило адекватность обученной модели (0.868 против 0.472 у дамми-модели).
    
Таким образом, модель Логистической регрессии была обучена и протестирована на тестовой выборке, показатель F1 удовлетворяет требованиям (0.765 > 0.75). 